# Phase 3


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

C:\Users\Alwaz\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Alwaz\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Alwaz\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Alwaz\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
gen = ImageDataGenerator()
batch_size = 2000

#Fetching Image ________

flow = gen.flow_from_directory("C:\\Users\\Alwaz\Desktop\\Gridcv dataset\\", color_mode="grayscale", target_size=(200, 200), batch_size=batch_size)
out = flow.next()

Found 2096 images belonging to 2 classes.


In [3]:
flow.class_indices

{'human images': 0, 'non human images': 1}

In [4]:
X,y = out
print(X.shape)
print(y.shape)
X = X.reshape(X.shape[0],-1)
y =out[1].argmax(axis=1)
print()

(2000, 200, 200, 1)
(2000, 2)



In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#  Reshaping _____

X_train=X_train.reshape(X_train.shape[0],-1)
X_test=X_test.reshape(X_test.shape[0],-1)

print("X_train shape :",X_train.shape)
print("y_train shape :",y_train.shape)
print("X_test shape :",X_test.shape)
print("y_test shape :",y_test.shape)

X_train shape : (1340, 40000)
y_train shape : (1340,)
X_test shape : (660, 40000)
y_test shape : (660,)


In [6]:
              #        Logistic Regression ______________

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score

grid={"C":np.logspace(-2,2,7),"penalty":["l1","l2"], "solver":["liblinear"], "tol":np.logspace(-2,2,4)},

logistic_regression=LogisticRegression(random_state=42)
log_reg_cv=GridSearchCV(logistic_regression,grid,cv=10)

log_reg_cv.fit(X_train, y_train)

print("accuracy: ", log_reg_cv.best_score_)


res=log_reg_cv.predict(X_test)

print(str(round(accuracy_score(y_test,res)*100,2)) + " % For LogisticRegression with Grid Search") 

accuracy:  0.8082089552238806
81.36 % For LogisticRegression with Grid Search


In [8]:
                   # KNN without GRID ________

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

classifier = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
classifier.fit(X_train, y_train)

pred_y_knn_without = classifier.predict(X_test)

print(str(round(accuracy_score(y_test,pred_y_knn_without)*100,2)) + " % For KNN without Grid Search") 

77.42 % For KNN without Grid Search


In [9]:
                #  SVC without GRIDsearch ______________________2
     
from sklearn.svm import SVC
class_svc = SVC(kernel='linear', random_state=0)
class_svc.fit(X_train, y_train)
y_pred_svc = class_svc.predict(X_test)

print(str(round(accuracy_score(y_test,y_pred_svc)*100,2)) + " % For SVC without Grid Search") # SVC

81.52 % For SVC with Grid Search


In [10]:
               # KNN with GRID ________________
    
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
        ('sc', StandardScaler()),     
        ('knn', KNeighborsClassifier(algorithm='brute')) 
    ])

params = {
        'knn__n_neighbors': [3, 5, 7, 9, 11] # usually odd numbers
    }

clf = GridSearchCV(estimator=pipe,           
                      param_grid=params, 
                      cv=5,
                      return_train_score=True) # Turn on cv train scores
clf.fit(X_train, y_train)

y_pred_knn=clf.predict(X_test)

print(str(round(accuracy_score(y_test,y_pred_knn)*100,2)) + " % For KNN with Grid Search") # knn

77.42 % For KNN with Grid Search


In [ ]:
          # Inputting Image for recognition _______
    
import tqdm
from keras.preprocessing import image

train_image = []
for i in (range(1,555)):
    try:
        img = image.load_img('C:\\Users\\Alwaz\\Desktop\\Gridcv dataset\\non human images\\t ({a}).jpeg'.format(a=i), target_size=(200,200),grayscale=True,interpolation='nearest')
        img = image.img_to_array(img)
        path = 'C:\\Users\\Alwaz\\Desktop\\Gridcv dataset\\non human images\\t ({a}).jpeg'.format(a=i)
        train_image.append(img)
    except Exception as e:
        print(e)
        continue
    r = np.array(img)
    r=r.reshape(1,-1)
    #print(r.shape)
    print(".")
    if clf.predict(r)==1:
        print(path)
    #print(clf.predict(r))
    
#print(r.shape)
#r=r.reshape(r.shape[0],-1)
#print(r.shape)
#print(path)
#print(clf.predict(r))

